In [1]:
#1
import numpy as np

class ScratchSVMClassifier():
    def __init__(self, num_iter, lr, kernel='linear', threshold=1e-5, verbose=False,
                 gamma=1, coef0=0, degree=3):
        self.iter = num_iter
        self.lr = lr
        self.kernel = kernel
        self.threshold = threshold
        self.verbose = verbose


        self.gamma = gamma
        self.coef0 = coef0
        self.degree = degree

    def _kernel_function(self, x1, x2):

        if self.kernel == 'linear':
            return np.dot(x1, x2.T)
        elif self.kernel == 'polynomial':
            return (self.gamma * np.dot(x1, x2.T) + self.coef0) ** self.degree
        else:
            raise ValueError(f"Unknown kernel: {self.kernel}")

In [2]:
    def _update_lagrange_multipliers(self, X, y, lam):

        m = X.shape[0]
        for i in range(m):

            kernel_values = self._kernel_function(X[i], X)
            summation = np.sum(lam * y * y[i] * kernel_values)
            lam_new = lam[i] + self.lr * (1 - summation)
            lam[i] = max(0, lam_new)
        return lam

In [3]:
    def _select_support_vectors(self, X, y, lam):

        support_vector_indices = np.where(lam > self.threshold)[0]
        self.n_support_vectors = len(support_vector_indices)
        self.index_support_vectors = support_vector_indices
        self.lam_sv = lam[support_vector_indices]
        self.X_sv = X[support_vector_indices]
        self.y_sv = y[support_vector_indices]

In [4]:
    def fit(self, X, y, X_val=None, y_val=None):
        m, n = X.shape

        lam = np.zeros(m)

        for it in range(self.iter):
            lam = self._update_lagrange_multipliers(X, y, lam)
            self._select_support_vectors(X, y, lam)

            if self.verbose and (it % 10 == 0 or it == self.iter - 1):
                print(f"Iteration {it+1}/{self.iter}, support vectors: {self.n_support_vectors}")

In [5]:
    def predict(self, X):

        m = X.shape[0]
        y_pred = np.zeros(m)

        for i in range(m):
            kernel_vals = self._kernel_function(X[i], self.X_sv)
            decision = np.sum(self.lam_sv * self.y_sv * kernel_vals)
            y_pred[i] = 1 if decision >= 0 else -1

        return y_pred.astype(int)